# Test dump
## Project: Bicycle node network loop analysis

This is a dump for testing code. It smells a bit.

## Parameters

In [ ]:
%run -i setup_parameters.py
load_data = True  # Set to False if data are huge and have already been loaded
debug = True  # Set to True for extra plots and verbosity
plt.style.use(PATH["parameters"] + "plotstyle.mplstyle")

In [ ]:
try:  # See if allloops_dict exists. If not, initialize. This allows running multiple scenarios. Here we run only scenario 2 though.
    allloops_dict
except NameError:
    allloops_dict = {}
    dfunified_scenarios = {}
allloops_dict[SCENARIOID] = {}

In [ ]:
print("Running scenario " + str(SCENARIOID) + " in " + STUDY_AREA)
for k, v in SCENARIO[SCENARIOID].items():
    print(k + ": " + str(v))

## Functions

In [ ]:
%run -i functions.py

# Testing snapping

In [ ]:
pois_orig = load_pois()
if load_data:
    if LOOP_LENGTH_BOUND:
        llb_string = "_maxlength" + str(LOOP_LENGTH_BOUND)
    else:
        llb_string = ""

    with open(
        PATH["data_out"]
        + "loopcensus_"
        + str(LOOP_NUMNODE_BOUND)
        + llb_string
        + ".pkl",
        "rb",
    ) as f:
        allloops = pickle.load(f)
        alllooplengths = pickle.load(f)
        allloopnumnodes = pickle.load(f)
        allloopmaxslopes = pickle.load(f)
        Gnx = pickle.load(f)
        LOOP_NUMNODE_BOUND = pickle.load(f)
        nodes_id = pickle.load(f)
        nodes_coords = pickle.load(f)
        numloops = pickle.load(f)
        faceloops = pickle.load(f)

# Create gdf and igraph versions
nodes, edges = momepy.nx_to_gdf(net=Gnx, points=True, lines=True)
nodes.set_crs(epsg=25832, inplace=True)
edges.set_crs(epsg=25832, inplace=True)
G = ig.Graph.from_networkx(Gnx)
G.summary()

Create a minimal graph with 3 links, each 1km long, somewhere in Denmark:

In [ ]:
edges.drop(edges.index, inplace=True)
new_edges = gpd.GeoDataFrame(
    {
        "has_facility": [False, False, False],
        "has_service": [False, False, False],
        "has_water": [False, False, False],
        "poi_diversity": [0, 0, 0],
        "has_attraction": [False, False, False],
        "node_start": [0, 1, 1],
        "node_end": [1, 2, 3],
        "edge_id": [0, 1, 2],
        "_igraph_index": [0, 1, 2],
        "max_slope": [0, 0, 0],
        "weight": [1000, 1000, 1000],
        "geometry": [
            LineString([(600000, 600000), (601000, 600000)]),
            LineString([(601000, 600000), (602000, 600000)]),
            LineString([(601000, 600000), (601000, 601000)]),
        ],
    }
)
edges = gpd.GeoDataFrame(pd.concat([edges, new_edges], ignore_index=True))
edges

Put a POI on the central node and check that it snaps to all links:

In [ ]:
pois_orig.drop(pois_orig.index, inplace=True)
new_pois = gpd.GeoDataFrame(
    {
        "category": ["facility"],
        "geometry": [
            Point([(601000, 600000)]),
        ],
    }
)
pois_orig = gpd.GeoDataFrame(pd.concat([pois_orig, new_pois], ignore_index=True))
pois_orig

In [ ]:
pois_snapped = snap_pois(pois_orig, edges)
# edges = update_poi_attributes(edges, pois_snapped)

In [ ]:
pois_snapped

In [ ]:
edges

In [ ]:
edges = update_poi_attributes(edges, pois_snapped)
edges